In [1]:
import os

os.chdir('..')
%pwd

'e:\\DataScienceProjects\\car-price-prediction'

In [2]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_dir: Path
    source_URL: str

In [3]:
# Configuration Manager
from carPricePrediction.constants import *
from carPricePrediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            dataset_dir = config.dataset_dir,
            source_URL = config.source_URL
        )

        return data_ingestion_config

In [4]:
# Data ingestion component
import os 
from kaggle.api.kaggle_api_extended import KaggleApi
from carPricePrediction.logging import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        # Connect to Kaggle API
        api = KaggleApi()
        api.authenticate()

        # Download the dataset
        api.dataset_download_files(
            dataset=self.config.source_URL,
            path=self.config.dataset_dir,
            unzip=True
        )

        logger.info("Dataset downloaded successfully.")

In [5]:
# Data ingestion training pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
except Exception as e:
    raise e

[2024-03-29 17:43:40,234: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-29 17:43:40,236: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-29 17:43:40,238: INFO: common: Created directory at: artifacts]
[2024-03-29 17:43:40,239: INFO: common: Created directory at: artifacts/data]
[2024-03-29 17:43:42,691: INFO: 4265816127: Dataset downloaded successfully.]
